# Total OS frequencies (over the entire region/time period)
This is summing the appropriate bins in the Tb-cpT histograms (below dashed and solid lines)

In [1]:
import pickle

import xarray as xr
import numpy as np


In [2]:
save_dir = "/home/b/b380887/cold-point-overshoot/plots/obs_paper/tb_cp_hists/"
pickle_dir = "/home/b/b380887/cold-point-overshoot/pickle_files/climo_tb_cp_hists//"


In [3]:
season = "DJF"


In [4]:
if season == "DJF":
    region_list = ["AMZ", "ECP", "IOS", "SPC"]
    year_list = [2007, 2008, 2009, 2010]
    offset_dict = {
        "AMZ": 9,
        "SPC": 10,
        "IOS": 9,
        "ECP": 10
    }

elif season == "JJA":
    region_list = ["AFR", "ECP", "WPC", "IOE"]
    year_list = [2007, 2008, 2009, 2010] 
    offset_dict = {
        "AFR": 10,
        "WPC": 11,
        "IOE": 8,
        "ECP": 8
    }
    
file_path = "/work/bb1153/b380887/big_obs_climo/{s}/".format(s=season)

years_sorted = list(sorted(year_list))
years_str = str(years_sorted[0]) + "-" + str(years_sorted[-1])
print(years_str)


2007-2010


In [13]:
hist_region_dict = {}

for region in region_list:
    hist_computed_list = [[]]*len(year_list)
    nan_len_list = [[]]*len(year_list)

    for i, year in enumerate(year_list):
        file_to_open = pickle_dir + "Tb-cpT_hist_dict_{s}{y}_{r}.pickle".format(s=season, y=year, r=region)
        with open(file_to_open, "rb") as handle:
            hist_dict_yr = pickle.load(handle)
        hist_computed_list[i] = hist_dict_yr["hist_computed"]
        nan_len_list[i] = hist_dict_yr["nan_len"]

    # sum across years
    nan_len = sum(nan_len_list)
    hist_computed = np.sum(hist_computed_list, axis=0)

    # take bins & edges from the last dict in list
    hist_dict = {
        "hist_computed": hist_computed, 
        "xedges": hist_dict_yr["xedges"], 
        "yedges": hist_dict_yr["yedges"], 
        "tb_bins": hist_dict_yr["tb_bins"], 
        "cpT_bins": hist_dict_yr["cpT_bins"],
        "nan_len": nan_len,
    }

    hist_region_dict[region] = hist_dict


In [9]:
def get_total_os_freq(hist_dict, offset, region):
    """ Get counts and frequencies of Tb < cold point and threshold
    """
    hist = hist_dict["hist_computed"]
    tb_edges = hist_dict["yedges"]
    cpT_edges = hist_dict["xedges"]
    xbin_means, ybin_means = (cpT_edges[:-1]+cpT_edges[1:])/2, (tb_edges[:-1]+tb_edges[1:])/2
    
    da = xr.DataArray(hist_dict["hist_computed"],
             dims=["cpT", "tb"], 
             coords={"cpT": xbin_means, "tb": ybin_means},
                  attrs={"region": region}
            )
    
    count_below_thresh = np.nansum(da.where(da.tb < (da.cpT+offset)).values)
    count_below_cp = np.nansum(da.where(da.tb < da.cpT).values)
    
    # nan_len is tb count, not the count on this histogram;
    # it was used to normalize the histogram in the plots, but here we use the
    # sum of the total histogram counts
    freq_below_thresh = count_below_thresh / da.sum().values
    freq_below_cp = count_below_cp / da.sum().values
    
    out_dict = {
        "region": region,
        "offset": offset,
        "count_below_thresh": count_below_thresh,
        "count_below_cp": count_below_cp,
        "freq_below_thresh": freq_below_thresh,
        "freq_below_cp": freq_below_cp,
        "thresh_def": "cold point bin mean + offset"
    }

    return out_dict


### Show the frequencies and save those dictionaries

In [12]:
# DJF
out_dict_djf = {}
for region in region_list:
    out_dict = get_total_os_freq(hist_region_dict[region], offset_dict[region], region)
    out_dict_djf[region] = out_dict
    freq_thresh = out_dict["freq_below_thresh"]*100.
    freq_cp = out_dict["freq_below_cp"]*100.

    # print("{r}: {t:.3f}%, {c:.4f}%".format(t=freq_thresh, c=freq_cp, r=region))
    print("{r}: {c}%".format(c=np.round(freq_cp, 4), r=region))

    out_file = pickle_dir + "total_os_freq_dict_{s}{y}_{r}.pickle".format(s=season, y=years_str, r=region)
    # with open(out_file, "wb") as handle:
    #     pickle.dump(out_dict, handle)


AMZ: 0.0078%
ECP: 0.0002%
IOS: 0.0038%
SPC: 0.0065%


In [71]:
# JJA
out_dict_jja = {}
for region in region_list:
    out_dict = get_total_os_freq(hist_region_dict[region], offset_dict[region], region)
    out_dict_jja[region] = out_dict
    freq_thresh = out_dict["freq_below_thresh"]*100.
    freq_cp = out_dict["freq_below_cp"]*100.

    # print("{r}: {t:.3f}%, {c:.4f}%".format(t=freq_thresh, c=freq_cp, r=region))
    print("{r}: {c}%".format(c=np.round(freq_cp, 3), r=region))

    out_file = pickle_dir + "total_os_freq_dict_{s}{y}_{r}.pickle".format(s=season, y=years_str, r=region)
    # with open(out_file, "wb") as handle:
    #     pickle.dump(out_dict, handle)


AFR: 0.024%
ECP: 0.002%
WPC: 0.015%
IOE: 0.004%


In [73]:
def compare_two_regions(r1, r2, out_dict, count=False):
    if count:
        print(out_dict[r1]["count_below_cp"] / out_dict[r2]["count_below_cp"])
    else:
        print(out_dict[r1]["freq_below_cp"] / out_dict[r2]["freq_below_cp"])
    

In [78]:
compare_two_regions("AMZ", "SPC", out_dict_djf)
compare_two_regions("AMZ", "IOS", out_dict_djf)
compare_two_regions("AMZ", "ECP", out_dict_djf)
compare_two_regions("SPC", "IOS", out_dict_djf)

1.207010434938332
2.0739934835273703
36.628570606792344
1.7182896050383638


In [79]:
compare_two_regions("AFR", "WPC", out_dict_jja)
compare_two_regions("AFR", "IOE", out_dict_jja)
compare_two_regions("AFR", "ECP", out_dict_jja)
compare_two_regions("WPC", "IOE", out_dict_jja)

1.579406569052873
6.371141620047548
11.214958658414067
4.0338831969453866
